## Process structured data
- Read and review data
- Extract necessary keys from json to column
- Save final df as file for loading

In [6]:
# Import packages

import numpy as np
import pandas as pd
import json
import time
import csv
from pandas.io.json import json_normalize
import os

In [9]:
# Read the data file
ksdf = pd.read_json('input_data_files/Kickstarter_2019-07-18T03_20_05_009Z.json', lines=True)

In [10]:
ksdf.head()

,table_id,robot_id,run_id,data
0,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1456501169, 'photo': {'key': 'assets/01..."
1,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1259045207, 'photo': {'key': 'assets/01..."
2,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 577497800, 'photo': {'key': 'assets/012..."
3,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 521645557, 'photo': {'key': 'assets/012..."
4,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 504193797, 'photo': {'key': 'assets/011..."


In [11]:
ksdf.shape

(212378, 4)

# Limit data to US and completed projects

In [12]:
def get_country(d):
    return d['country']

ksdf['country'] = ksdf['data'].apply(lambda x: get_country(x))

In [13]:
def get_state(d):
    return d['state']

ksdf['state'] = ksdf['data'].apply(lambda x: get_state(x))

In [14]:
ksdf.head()

,table_id,robot_id,run_id,data,country,state
0,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1456501169, 'photo': {'key': 'assets/01...",US,canceled
1,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1259045207, 'photo': {'key': 'assets/01...",ES,failed
2,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 577497800, 'photo': {'key': 'assets/012...",US,canceled
3,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 521645557, 'photo': {'key': 'assets/012...",US,successful
4,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 504193797, 'photo': {'key': 'assets/011...",US,failed


In [15]:
ksdf_pared = ksdf.copy()[(ksdf['country'] == 'US') & (ksdf['state'].isin(['failed', 'successful']))]

ksdf_pared.shape

(139246, 6)

In [45]:
ksdf_pared['state'].unique()

array(['successful', 'failed'], dtype=object)

In [83]:
type(ksdf_pared['data'])

pandas.core.series.Series

In [44]:
ksdf_pared.shape

(196491, 6)

In [32]:
ksdf_pared.sample(10)

,table_id,robot_id,run_id,data,country,state,projecturls
20795,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1316835288, 'photo': {'key': 'assets/01...",US,failed,https://www.kickstarter.com/projects/145624496...
122007,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1393554486, 'photo': {'key': 'assets/01...",US,failed,https://www.kickstarter.com/projects/154906855...
199181,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 905679522, 'photo': {'key': 'assets/024...",US,successful,https://www.kickstarter.com/projects/267974484...
47666,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 124290710, 'photo': {'key': 'assets/023...",US,failed,https://www.kickstarter.com/projects/jendaresu...
79701,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1977273541, 'photo': {'key': 'assets/01...",US,failed,https://www.kickstarter.com/projects/gaufamily...
179383,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1760144456, 'photo': {'key': 'assets/01...",US,failed,https://www.kickstarter.com/projects/104821716...
89965,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 588266637, 'photo': {'key': 'assets/012...",US,failed,https://www.kickstarter.com/projects/115313888...
70930,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1294644168, 'photo': {'key': 'assets/02...",US,successful,https://www.kickstarter.com/projects/artwalks/...
32796,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 350729554, 'photo': {'key': 'assets/012...",US,successful,https://www.kickstarter.com/projects/139295367...
105585,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1408548907, 'photo': {'key': 'assets/01...",US,failed,https://www.kickstarter.com/projects/200842709...


# Extracting dictionary keys to columns
- need to iterate over dictionaries in data column
- grab 'project' values
- convert 'project' to column
- concat to df

In [17]:
# Test extraction of a single key to column

project_col = []

for i in ksdf_pared.data:
    project_url = i['urls']['web']['project']
    project_col.append(project_url)

In [18]:
len(project_col)

139246

In [19]:
project_col[-1]

'https://www.kickstarter.com/projects/61747643/maranatha-short-film?ref=discovery_category_newest'

In [20]:
ksdf_pared['projecturls'] = project_col

In [61]:
# Define a function that checks if the key exists
def check_key(dictionary, key):
    if key in dictionary.keys():
        return dictionary[key]
    else:
        return ""

In [51]:
# Test function
# check_key(ksdf_pared.loc[655, 'data'], 'id')

1388506064

In [111]:
# Define and iterate over all the 1st level keys to be extracted to columns

d_id = []
d_name = []
d_blurb =[]
d_goal =[]
d_pledged =[]
d_currencysym =[]
d_deadline = []
d_statechangedat =[]
d_createdat = []
d_launchedat = []
d_staffpick = []
d_backerscnt = []
d_location = []
d_category = []
d_spotlight = []

    
for i in ksdf_pared.data:
    
    data_id = check_key(i, 'id')
    d_id.append(data_id)
    
    name = check_key(i, 'name')
    d_name.append(name)
    
    blurb = check_key(i, 'blurb')
    d_blurb.append(blurb)
    
    goal = check_key(i, 'goal')
    d_goal.append(goal)
    
    pledged = check_key(i, 'pledged')
    d_pledged.append(pledged)
    
    dollarsign = check_key(i, 'currency_symbol')
    d_currencysym.append(dollarsign)
    
    deadline = check_key(i, 'deadline')
    d_deadline.append(deadline)
                   
    state_changed_at = check_key(i, 'state_changed_at')
    d_statechangedat.append(state_changed_at)
                   
    created_at = check_key(i, 'created_at')
    d_createdat.append(created_at)
                   
    launched_at = check_key(i, 'launched_at')
    d_launchedat.append(launched_at)
                   
    staff_pick = check_key(i, 'staff_pick')
    d_staffpick.append(staff_pick)
                   
    backers_count = check_key(i, 'backers_count')
    d_backerscnt.append(backers_count)
                   
    location = check_key(i, 'location')
    d_location.append(location)
                   
    category = check_key(i, 'category')
    d_category.append(category)
                   
    spotlight = check_key(i, 'spotlight')
    d_spotlight.append(spotlight)
    
       

In [112]:
# Conduct similar iteration for nested keys
location_id = []
location_name = []
location_state = []
category_id = []
category_name = []
category_slug = []

# For each nested key, check that it's a dictionary and append the value, if not return empty

# For nested location data
for i in d_location:
    if type(i) == dict:
        locationid = check_key(i, 'id')
        location_id.append(locationid)
    else:
        location_id.append("")
        
for i in d_location:
    if type(i) == dict:
        locationname = check_key(i, 'name')
        location_name.append(locationname)
    else:
        location_name.append("")
        
for i in d_location:
    if type(i) == dict:
        locationstate = check_key(i, 'state')
        location_state.append(locationstate)
    else:
        location_state.append("")

# For nested category data
for i in d_category:
    if type(i) == dict:
        categoryid = check_key(i, 'id')
        category_id.append(categoryid)
    else:
        category_id.append("")
        
for i in d_category:
    if type(i) == dict:
        categoryname = check_key(i, 'name')
        category_name.append(categoryname)
    else:
        category_name.append("")

for i in d_category:
    if type(i) == dict:
        categoryslug = check_key(i, 'slug')
        category_slug.append(categoryslug)
    else:
        category_slug.append("")
        

In [113]:
# Check to see that all the lists have the same length

lists = [d_id,
d_name,
d_blurb ,
d_goal,
d_pledged,
d_currencysym ,
d_deadline,
d_statechangedat,
d_createdat,
d_launchedat,
d_staffpick,
d_backerscnt,
d_category,
d_spotlight,
location_id,
location_name ,
location_state ,
category_id,
category_name ,
category_slug
        ]
it = iter(lists)
the_len = len(next(it))

if not all(len(l) == the_len for l in it):
    raise ValueError('not all lists have same length!') 


    

In [114]:
# Make a copy of dataframe to be safe

ksdf_pcopy = ksdf_pared.copy()

In [115]:
ksdf_pcopy.sample(10)

,table_id,robot_id,run_id,data,country,state,projecturls
184644,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 477263730, 'photo': {'key': 'assets/011...",US,successful,https://www.kickstarter.com/projects/158626959...
203639,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1909527085, 'photo': {'key': 'assets/01...",US,successful,https://www.kickstarter.com/projects/651841039...
208478,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1513307111, 'photo': {'key': 'assets/01...",US,successful,https://www.kickstarter.com/projects/166447266...
35090,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1446656383, 'photo': {'key': 'assets/01...",US,successful,https://www.kickstarter.com/projects/165210922...
64007,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1765235207, 'photo': {'key': 'assets/01...",US,successful,https://www.kickstarter.com/projects/920311578...
147726,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 2056250552, 'photo': {'key': 'assets/02...",US,successful,https://www.kickstarter.com/projects/brandonfi...
84731,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 780653021, 'photo': {'key': 'assets/021...",US,failed,https://www.kickstarter.com/projects/429078619...
192428,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1050129003, 'photo': {'key': 'assets/01...",US,failed,https://www.kickstarter.com/projects/214652118...
100325,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 186726844, 'photo': {'key': 'assets/016...",US,failed,https://www.kickstarter.com/projects/141821046...
124153,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1031555498, 'photo': {'key': 'assets/01...",US,successful,https://www.kickstarter.com/projects/372333499...


In [116]:
ksdf_pcopy.shape

(139246, 7)

In [117]:
# Concat the lists to a copy of the data frame

ksdf_pcopy['data_id'] = d_id
ksdf_pcopy['d_name'] = d_name
ksdf_pcopy['d_blurb'] = d_blurb
ksdf_pcopy['d_goal'] = d_goal
ksdf_pcopy['d_pledged'] = d_pledged
ksdf_pcopy['d_dollar'] = d_currencysym
ksdf_pcopy['d_deadline'] = d_deadline
ksdf_pcopy['d_statechangedat'] = d_statechangedat
ksdf_pcopy['d_createdat'] = d_createdat
ksdf_pcopy['d_launchedat'] = d_launchedat
ksdf_pcopy['d_staffpick'] = d_staffpick
ksdf_pcopy['d_nobackers'] = d_backerscnt
ksdf_pcopy['d_spotlight'] = d_spotlight
ksdf_pcopy['location_id'] = location_id
ksdf_pcopy['loc_name'] = location_name
ksdf_pcopy['loc_state'] = location_state
ksdf_pcopy['category_id'] = category_id
ksdf_pcopy['cat_name'] = category_name
ksdf_pcopy['cat_slug'] = category_slug


In [118]:
ksdf_pcopy.sample(20)

,table_id,robot_id,run_id,data,country,state,projecturls,data_id,d_name,d_blurb,...,d_launchedat,d_staffpick,d_nobackers,d_spotlight,location_id,loc_name,loc_state,category_id,cat_name,cat_slug
97779,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1308436596, 'photo': {'key': 'assets/01...",US,failed,https://www.kickstarter.com/projects/124572182...,1308436596,The COSMIC band,"The GREAT,UNIQUE COSMIC NeoSOULjazz COSMIC ban...",...,1336929594,False,0,False,2451822,Milwaukee,WI,41,Jazz,music/jazz
24100,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1931226286, 'photo': {'key': 'assets/01...",US,successful,https://www.kickstarter.com/projects/550011749...,1931226286,Solo album by Thom Ayres (of Arcanta),A solo album from the man behind the music of ...,...,1351111554,False,34,True,2379574,Chicago,IL,42,Pop,music/pop
92833,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1666227891, 'photo': {'key': 'assets/02...",US,successful,https://www.kickstarter.com/projects/133110502...,1666227891,Ellie Marches On,This book is a gentle way to introduce the imp...,...,1554739922,True,74,True,2486982,St. Louis,MO,46,Children's Books,publishing/children's books
156533,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1794155840, 'photo': {'key': 'assets/01...",US,successful,https://www.kickstarter.com/projects/ca7ch/ca7...,1794155840,CA7CH Lightbox,"Designed for smart phones, Lightbox lets you s...",...,1403013782,False,1631,True,2459115,New York,NY,341,Wearables,technology/wearables
31254,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 634737497, 'photo': {'key': 'assets/012...",US,successful,https://www.kickstarter.com/projects/137095375...,634737497,Help me buy a loom!,I want to start making hand woven scarves and ...,...,1446847091,False,12,True,2508349,Tualatin,OR,355,Weaving,crafts/weaving
30219,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 1873993845, 'photo': {'key': 'assets/02...",US,successful,https://www.kickstarter.com/projects/terrencee...,1873993845,WICKED ENIGMA,LGBTQ Murder Mystery Webseries,...,1530029887,False,124,True,2442047,Los Angeles,CA,33,Webseries,film & video/webseries
39569,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 596591038, 'photo': {'key': 'assets/023...",US,successful,https://www.kickstarter.com/projects/oddwhimsy...,596591038,Intuitive abstract painting to melodious music,"creating, for you, an original and delightfull...",...,1546437843,True,78,True,2427032,Indianapolis,IN,54,Mixed Media,art/mixed media
119794,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 834491219, 'photo': {'key': 'assets/011...",US,successful,https://www.kickstarter.com/projects/679109791...,834491219,Carleigh Nesbit's New Album 'Come Out of the K...,Carleigh Nesbit is going back into the studio ...,...,1425688522,False,170,True,2378489,Charlottesville,VA,37,Country & Folk,music/country & folk
112278,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 69767453, 'photo': {'key': 'assets/011/...",US,successful,https://www.kickstarter.com/projects/attnwhre/...,69767453,"Attention, Whore is poor, but for less than a ...","Hey guys, we're going on tour. We really need ...",...,1368484991,False,23,True,2452078,Minneapolis,MN,43,Rock,music/rock
172770,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,"{'id': 2130212660, 'photo': {'key': 'assets/02...",US,successful,https://www.kickstarter.com/projects/threadint...,2130212660,A Better Backpack,Get a better all-day backpack designed to look...,...,1533732830,True,3533,True,2473224,Pittsburgh,PA,28,Product Design,design/product design


In [119]:
# Save df to csv
ksdf_pcopy.to_csv('ks_structured.csv', sep='\t')

In [123]:
# remove the 'data' json field from the df to preserve format when saving
# save as new df
ksdf_cnodata = ksdf_pcopy.drop('data', 1)

In [124]:
ksdf_cnodata.sample(3)

,table_id,robot_id,run_id,country,state,projecturls,data_id,d_name,d_blurb,d_goal,...,d_launchedat,d_staffpick,d_nobackers,d_spotlight,location_id,loc_name,loc_state,category_id,cat_name,cat_slug
5366,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,US,successful,https://www.kickstarter.com/projects/143783311...,58167066,Mobile Radio/Recording Studio in Medellin Colo...,Our aim is to work with the existing network o...,10000.0,...,1308679781,False,66,True,368150,Medellin,Antioquia,239,Radio & Podcasts,publishing/radio & podcasts
28828,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,US,successful,https://www.kickstarter.com/projects/sisl/hot-...,1813895592,Hot Pursuit: How Chilies Set the World on Fire,"Until 1493, chili peppers had never left the A...",25000.0,...,1544537532,True,169,True,12589335,Brooklyn,NY,30,Documentary,film & video/documentary
4353,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,US,successful,https://www.kickstarter.com/projects/christope...,12236287,Christoper Noyes Summer Tour,I will be going on some tours this summer in s...,900.0,...,1338392501,False,15,True,2372503,Bushnell,IL,44,World Music,music/world music


In [140]:
# Save the final df to csv
ksdf_cnodata.to_csv('ks_struct_nodata.csv', sep='\t', index = False)

In [141]:
# Perform a test read to ensure the df can be read prior to loading to postgres
test = pd.read_csv('ks_struct_nodata.csv', sep='\t')

In [142]:
test.head(5)

,table_id,robot_id,run_id,country,state,projecturls,data_id,d_name,d_blurb,d_goal,...,d_launchedat,d_staffpick,d_nobackers,d_spotlight,location_id,loc_name,loc_state,category_id,cat_name,cat_slug
0,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,US,successful,https://www.kickstarter.com/projects/184955320...,521645557,How to Breathe Underwater/In the Black Box -- ...,A surrealist's guide to navigating the dark de...,1500.0,...,1465488766,False,44,True,2356940.0,Athens,GA,42,Pop,music/pop
1,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,US,failed,https://www.kickstarter.com/projects/760226815...,504193797,Backpackable one-person high resolution UAV ma...,Continue to develop a user friendly one-person...,10000.0,...,1397105848,False,3,False,2441116.0,Logan,UT,336,Flight,technology/flight
2,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,US,failed,https://www.kickstarter.com/projects/143006059...,1667139345,LOVE STORY,LOVE STORY IS ABOUT THE REAL LOVE THAT PEOPLE ...,2500.0,...,1376512056,False,5,False,2503713.0,Tallahassee,FL,44,World Music,music/world music
3,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,US,failed,https://www.kickstarter.com/projects/533614462...,1389221860,Detroit Revue Concert,Together we can bring a star back on the map o...,35500.0,...,1375666413,False,0,False,2391585.0,Detroit,MI,44,World Music,music/world music
4,Kickstarter,Kickstarter,Kickstarter_2019-07-18T03_20_05_009Z,US,successful,https://www.kickstarter.com/projects/131995875...,1730678034,"Let's Commit (to) a MURDER, SHE WROTE Album of...",Songwriter Joel B. New is creating his first-e...,6500.0,...,1457360083,False,165,True,2459115.0,New York,NY,42,Pop,music/pop
